### Preprocessing of babi all

In [ ]:
!wget -nc http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz .
!tar -C .  -zxvf  ./tasks_1-20_v1-2.tar.gz 
!rm tasks_1-20_v1-2.tar.gz

In [ ]:
import os

directories = ["babi1", "babi2", "babi3"]

def dirmaker(list):

    for item in list:

        try:

            os.makedirs(item)

        except:

            print("Folder already exists")
            
dirmaker(directories)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

keys = ['train', 'test']

In [ ]:
def _parse(file):
    data, story = [], []
    for line in open(file).readlines():
        tid, text = line.rstrip('\n').split(' ', 1)
        if tid == '1':
            story = []
        # sentence
        if text.endswith('.'):
            story.append(text[:-1])
        # question
        else:
            # remove any leading or trailing whitespace after splitting
            query, answer, supporting = (x.strip() for x in text.split('\t'))
            substory = [x for x in story if x]
            data.append((substory, query[:-1], answer))    # remove '?'
            story.append("")
    return data

In [ ]:
tasks = ['qa1_single-supporting-fact_', 'qa2_two-supporting-facts_', 'qa3_three-supporting-facts_']
data = {}
for t in tasks :
    data[t] = {}
    for k in keys :
        data[t][k] = list(zip(*_parse('./tasks_1-20_v1-2/en-10k/' + t + k + '.txt')))

In [ ]:
p, q, a = {}, {}, {}

for k in keys :
    p[k], q[k], a[k] = {}, {}, {}
    for t in tasks :
        p[k][t] = [" . ".join(x) for x in data[t][k][0]]
        q[k][t] = data[t][k][1]
        a[k][t] = data[t][k][2]

In [ ]:
import pandas as pd
import os

file_paths = {}

for i,t in enumerate(tasks):
    df_paragraphs = []
    df_questions = []
    df_answers = []
    df_exp_splits = []

    for k in keys :
        df_paragraphs += p[k][t]
        df_questions += q[k][t]
        df_answers += a[k][t]
        df_exp_splits += [k] * len(p[k][t])

    df = {'paragraph' : df_paragraphs, 'question' : df_questions, 'answer' : df_answers, 'exp_split' : df_exp_splits}
    df = pd.DataFrame(df)
    
    from sklearn.model_selection import train_test_split
    train_idx, dev_idx = train_test_split(df.index[df.exp_split == 'train'], test_size=0.15, random_state=16377)
    df.loc[dev_idx, 'exp_split'] = 'dev'
    
    file_paths["babi" + str(i + 1)] = './babi' + str(i+1) + '/data_splits.csv'
    
    df.to_csv('babi' + str(i+1) + '/data_splits.csv', index=False)

In [ ]:
import pickle
from preprocess_qa import extract_vocabulary_, text_to_seq, DataHolder_QA


c = 0

for task, filepath in file_paths.items():
    
    babi_ = pd.read_csv(filepath)
    
    ix_2_answers = dict(enumerate(set(babi_.answer)))
    answers_2_ix = {v:k for k,v in ix_2_answers.items()}
    
    word_to_ix = extract_vocabulary_(min_df = 1, dataframe = babi_)
    ix_to_word = {v:k for k,v in word_to_ix.items()}
    
    babi_["paragraph"] = babi_["paragraph"].apply(lambda x: "<SOS> " + x + " <EOS>")
    babi_["question"] = babi_["question"].apply(lambda x: "<SOS> " + x + " <EOS>")
    
    babi_["paragraph"] = babi_["paragraph"].apply(lambda x: text_to_seq(x, word_to_ix))
    babi_["question"] = babi_["question"].apply(lambda x: text_to_seq(x, word_to_ix))
    babi_["answer"] = babi_["answer"].apply(lambda x: text_to_seq(x, word_to_ix))#answers_2_ix, True))
    
    train_ix = babi_[babi_.exp_split == "train"][["paragraph","question","answer"]].values
    dev_ix = babi_[babi_.exp_split == "dev"][["paragraph","question", "answer"]].values
    test_ix = babi_[babi_.exp_split == "test"][["paragraph","question", "answer"]].values
    
    data = DataHolder_QA(train_ix, dev_ix, test_ix, word_to_ix, answers_2_ix)
    pickle.dump(data, open(filepath.split("/")[1] + "/data.p", "wb"))
    
    c+=1
    
    print(len(word_to_ix))

In [ ]:
[ix_to_word[w] for w in train_ix[0][1]]

In [ ]:
!rm -r tasks_1-20_v1-2/